Curve fitting
=============

Curve fitting in ERLabPy largely relies on [lmfit](https://lmfit.github.io/lmfit-py/). Along with some convenient models for common fitting tasks, ERLabPy provides a powerful accessor that streamlines curve fitting on multidimensional xarray objects.

ERLabPy also provides optional integration of lmfit models with [iminuit ](https://github.com/scikit-hep/iminuit), which is a Python interface to the [Minuit C++ library ](https://root.cern.ch/doc/master/Minuit2Page.html) developed at CERN.

In this tutorial, we begin with some convenient functions that ERLabPy provides for common tasks such as Fermi edge fitting. Next, we will start with the basics of curve fitting using lmfit, introduce some models that are available in ERLabPy, and demonstrate curve fitting with the {meth}`modelfit <erlab.accessors.fit.ModelFitDataArrayAccessor.__call__>` accessor to fit multidimensional xarray objects. Finally, we will show how to use [iminuit](https://github.com/scikit-hep/iminuit) with lmfit models.


In [ ]:
import matplotlib.pyplot as plt
import erlab.plotting as eplt
import erlab.analysis as era
from erlab.io.exampledata import generate_gold_edge

plt.rcParams["figure.constrained_layout.use"] = True

(fermi edge fitting)=

## Fermi edge fitting

Functions related to the Fermi edge are available in {mod}`erlab.analysis.gold`. To fit a polynomial to a Fermi edge, you can use {func}`erlab.analysis.gold.poly`.


:::{hint}

The interactive Fermi edge fitting tool {func}`erlab.interactive.goldtool` can be used to generate the code below interactively.

:::

In [ ]:
gold = generate_gold_edge(temp=100, seed=1)

result = era.gold.poly(
    gold,
    angle_range=(-15, 15),
    eV_range=(-0.2, 0.2),
    temp=100.0,
    vary_temp=False,
    degree=2,
    plot=True,
)

The resulting polynomial can be used to correct the Fermi edge with {func}`erlab.analysis.gold.correct_with_edge`:

In [ ]:
era.gold.correct_with_edge(gold, result).qplot(cmap="Greys")
eplt.fermiline()

## Curve fitting with `lmfit`

In this section, we will cover basic curve fitting using [lmfit](https://lmfit.github.io/lmfit-py/). If you are familiar with the library, you can [skip to the next section](pre-defined-models).

In [ ]:
import erlab.analysis as era
import erlab.plotting as eplt
import numpy as np
import xarray as xr

In [ ]:
%config InlineBackend.figure_formats = ["svg", "pdf"]
plt.rcParams["figure.dpi"] = 96
plt.rcParams["image.cmap"] = "viridis"
plt.rcParams["figure.figsize"] = eplt.figwh(wscale=1.2, fixed_height=False)

nb_execution_mode = "cache"

Let's start by defining a model function and the data to fit.

In [ ]:
def poly1(x, a, b):
    return a * x + b


# Generate some toy data
x = np.linspace(0, 10, 20)
y = poly1(x, 1, 2)

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(1)
yerr = np.full_like(x, 0.5)
y = rng.normal(y, yerr)

### Fitting a simple function

A lmfit model can be created by calling {class}`lmfit.Model` class with the model function and the independent variable(s) as arguments.

In [ ]:
import lmfit

model = lmfit.Model(poly1)
params = model.make_params(a=1.0, b=2.0)
result = model.fit(y, x=x, params=params, weights=1 / yerr)

result.plot()
result

By passing dictionaries to `make_params`, we can set the initial values of the parameters and also set the bounds for the parameters.

In [ ]:
model = lmfit.Model(poly1)
params = model.make_params(
    a={"value": 1.0, "min": 0.0},
    b={"value": 2.0, "vary": False},
)
result = model.fit(y, x=x, params=params, weights=1 / yerr)
_ = result.plot()

`result` is a {class}`lmfit.model.ModelResult` object that contains the results of the fit. The best-fit parameters can be accessed through the `result.params` attribute.

:::{note}

Since all weights are the same in this case, it has little effect on the fit results. However, if we are confident that we have a good estimate of `yerr`, we can pass `scale_covar=True` to the `fit` method to obtain accurate uncertainties.

:::

In [ ]:
result.params

In [ ]:
result.params["a"].value, result.params["a"].stderr

The parameters can also be retrieved in a form that allows easy error propagation calculation, enabled by the [uncertainties](https://github.com/lmfit/uncertainties) package.

In [ ]:
a_uvar = result.uvars["a"]
print(a_uvar)
print(a_uvar**2)

### Fitting with composite models

Before fitting, let us generate a Gaussian peak on a linear background.

In [ ]:
# Generate toy data
x = np.linspace(0, 10, 50)
y = -0.1 * x + 2 + 3 * np.exp(-((x - 5) ** 2) / (2 * 1**2))

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(5)
yerr = np.full_like(x, 0.3)
y = rng.normal(y, yerr)

# Plot the data
plt.errorbar(x, y, yerr, fmt="o")

A composite model can be created by adding multiple models together.

In [ ]:
from lmfit.models import GaussianModel, LinearModel

model = GaussianModel() + LinearModel()
params = model.make_params(slope=-0.1, center=5.0, sigma={"value": 0.1, "min": 0})
params

In [ ]:
result = model.fit(y, x=x, params=params, weights=1 / yerr)
result.plot()
result

How about multiple gaussian peaks? Since the parameter names overlap between the models, we must use the `prefix` argument to distinguish between them.

In [ ]:
model = GaussianModel(prefix="p0_") + GaussianModel(prefix="p1_") + LinearModel()
model.make_params()

For more information, see the [lmfit documentation](https://lmfit.github.io/lmfit-py/fitting.html).

(pre-defined-models)=

## Fitting with pre-defined models

Creating composite models with different prefixes every time can be cumbersome, so ERLabPy provides some pre-defined models in {mod}`erlab.analysis.fit.models`.


### Fitting multiple peaks

One example is {class}`MultiPeakModel <erlab.analysis.fit.models.MultiPeakModel>`, which works like a composite model of multiple Gaussian or Lorentzian peaks.

By supplying keyword arguments, you can specify the number of peaks, their shapes, whether to multiply with a Fermi-Dirac distribution, the shape of the background, and whether to convolve the result with experimental resolution.

For a detailed explanation of all the arguments, see its {class}`documentation <erlab.analysis.fit.models.MultiPeakModel>`.

The model can be constructed as follows:

In [ ]:
model = era.fit.models.MultiPeakModel(
    npeaks=1, peak_shapes=["gaussian"], fd=False, background="linear", convolve=False
)
params = model.make_params(p0_center=5.0, p0_width=0.2, p0_height=3.0)
params

In [ ]:
result = model.fit(y, x=x, params=params, weights=1 / yerr)
_ = result.plot()

We can also plot components.

In [ ]:
comps = result.eval_components()
plt.errorbar(x, y, yerr, fmt="o", zorder=-1, alpha=0.3)
plt.plot(x, result.eval(), label="Best fit")
plt.plot(x, comps["1Peak_p0"], "--", label="Peak")
plt.plot(x, comps["1Peak_bkg"], "--", label="Background")
plt.legend()

Now, let us try fitting MDCs cut from simulated data with multiple Lorentzian peaks, convolved with a common instrumental resolution.

In [ ]:
from erlab.io.exampledata import generate_data

data = generate_data(seed=1).T
cut = data.qsel(ky=0.3)
cut.qplot(colorbar=True)

In [ ]:
mdc = cut.qsel(eV=0.0)
mdc.qplot()

First, we define the model and set the initial parameters.

In [ ]:
model = era.fit.models.MultiPeakModel(
    npeaks=2, peak_shapes=["lorentzian"], fd=False, background="linear", convolve=True
)

params = model.make_params(
    p0_center=-0.5,
    p1_center=0.5,
    p0_width=0.03,
    p1_width=0.03,
    lin_bkg={"value": 0.0, "vary": False},
    const_bkg=0.0,
    resolution=0.03,
)
params

Then, we can fit the model to the data:

In [ ]:
result = model.fit(mdc, x=mdc.kx, params=params)
result.plot()
result



## Fitting `xarray` objects

ERLabPy provides accessors for xarray objects that allows you to fit data with lmfit models: {meth}`xarray.DataArray.modelfit <erlab.accessors.fit.ModelFitDataArrayAccessor.__call__>` or {meth}`xarray.Dataset.modelfit <erlab.accessors.fit.ModelFitDatasetAccessor.__call__>`, depending on whether you want to fit a single DataArray or multiple DataArrays in a Dataset. The accessor returns a {class}`xarray.Dataset` including the best-fit parameters and the fit statistics. The example below illustrates how to use the accessor to conduct the MDC fit demonstrated in the previous section.

:::{hint}

The syntax of the accessors are similar to the xarray methods {meth}`xarray.DataArray.curvefit` and {meth}`xarray.Dataset.curvefit`.

:::

In [ ]:
result_ds = mdc.modelfit("kx", model, params=params)
result_ds

Let's take a closer look at data variables in the resulting Dataset.

- `modelfit_results` contains the underlying {class}`lmfit.model.ModelResult` object from the fit.
- `modelfit_coefficients` and `modelfit_stderr` contain the best-fit coefficients and their errors, respectively.
- `modelfit_stats` contains the [goodness-of-fit statistics](https://lmfit.github.io/lmfit-py/fitting.html#goodness-of-fit-statistics).

It may not be immediately obvious how this is useful, but the true power of the accessor comes from its ability to utilize xarray's powerful broadcasting capabilities described in the next section.

### Fitting across multiple dimensions

:::{note}

There is a dedicated module for Fermi edge fitting and correction, described [here](fermi edge fitting). The following example is for illustrative purposes.

:::

Suppose you have to fit a single model to multiple data points across some dimension, or even multiple dimensions. The accessor can handle this with ease.

Let's demonstrate this with a simulated cut that represents a curved Fermi edge at 100 K, with an energy broadening of 20 meV.

In [ ]:
from erlab.io.exampledata import generate_gold_edge

gold = generate_gold_edge(temp=100, Eres=0.02, seed=1)
gold.qplot(cmap="Greys")

We first select ± 0.2 eV around the Fermi level and fit the model across the energy
axis for every EDC.

In [ ]:
gold_selected = gold.sel(eV=slice(-0.2, 0.2))
result_ds = gold_selected.modelfit(
    "eV",
    era.fit.models.FermiEdgeModel(),
    params={"temp": {"value": 100.0, "vary": False}},
    guess=True,
)
result_ds

Notice how the data variables in the resulting Dataset now depend on the coordinate
`alpha`. Let's plot the center of the edge as a function of angle!

In [ ]:
gold.qplot(cmap="Greys")
plt.errorbar(
    gold_selected.alpha,
    result_ds.modelfit_coefficients.sel(param="center"),
    result_ds.modelfit_stderr.sel(param="center"),
    fmt=".",
)

Fitting is not limited to 1D models. The following example demonstrates a global fit to the cut with a multidimensional model. First, we normalize the data with the averaged intensity of each EDC and then fit the data to {class}`FermiEdge2dModel <erlab.analysis.fit.models.FermiEdge2dModel>`.

In [ ]:
gold_norm = gold_selected / gold_selected.mean("eV")
result_ds = gold_norm.T.modelfit(
    coords=["eV", "alpha"],
    model=era.fit.models.FermiEdge2dModel(),
    params={"temp": {"value": 100.0, "vary": False}},
    guess=True,
)
result_ds

Let's plot the fit results and the residuals.

In [ ]:
best_fit = result_ds.modelfit_best_fit.transpose(*gold_norm.dims)

fig, axs = eplt.plot_slices(
    [gold_norm, best_fit, best_fit - gold_norm],
    figsize=(4, 5),
    cmap=["viridis", "viridis", "bwr"],
    norm=[plt.Normalize(), plt.Normalize(), eplt.CenteredPowerNorm(1.0, vcenter=0)],
    colorbar="all",
    hide_colorbar_ticks=False,
    colorbar_kw={"width": 7},
)
eplt.set_titles(axs, ["Data", "FermiEdge2dModel", "Residuals"])

### Providing multiple initial guesses

You can also provide different initial guesses and bounds for different coordinates. To demonstrate, let's create some data containing multiple Gaussian peaks, each with a different center.

In [ ]:
# Define angle coordinates for 2D data
alpha = np.linspace(-5.0, 5.0, 100)
beta = np.linspace(-1.0, 1.0, 3)

# Center of the peaks along beta
center = np.array([-2.0, 0.0, 2.0])[:, np.newaxis]

# Gaussian peak on a linear background
y = -0.1 * alpha + 2 + 3 * np.exp(-((alpha - center) ** 2) / (2 * 1**2))

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(5)
yerr = np.full_like(y, 0.05)
y = rng.normal(y, yerr)

# Construct DataArray
darr = xr.DataArray(y, dims=["beta", "alpha"], coords={"beta": beta, "alpha": alpha})
darr.qplot()

We can provide different initial guesses for the peak positions along the `beta` dimension by passing a dictionary of DataArrays to the `params` argument.

In [ ]:
result_ds = darr.modelfit(
    coords="alpha",
    model=GaussianModel() + LinearModel(),
    params={
        "center": xr.DataArray([-2, 0, 2], coords=[darr.beta]),
        "slope": -0.1,
    },
)
result_ds

Let's overlay the fitted peak positions on the data.

In [ ]:
result_ds.modelfit_data.qplot()
result_center = result_ds.sel(param="center")
plt.plot(result_center.modelfit_coefficients, result_center.beta, ".-")

The same can be done with all parameter attributes that can be passed to {func}`lmfit.create_params` (e.g., `vary`, `min`, `max`, etc.). For example:

In [ ]:
result_ds = darr.modelfit(
    coords="alpha",
    model=GaussianModel() + LinearModel(),
    params={
        "center": {
            "value": xr.DataArray([-2, 0, 2], coords=[darr.beta]),
            "min": -5.0,
            "max": xr.DataArray([0, 2, 5], coords=[darr.beta]),
        },
        "slope": -0.1,
    },
)
result_ds


The accessor works with any `lmfit` model, including background models from
[lmfitxps](https://lmfitxps.readthedocs.io/). If you have
[lmfitxps](https://lmfitxps.readthedocs.io/) installed, you can use the `ShirleyBG`
model to iteratively fit a Shirley background to the data:
```python
from lmfitxps.models import ShirleyBG
from lmfit.models import GaussianModel

darr.modelfit("alpha", GaussianModel() + ShirleyBG())
```

### Visualizing fits

:::{note}

If you are viewing this documentation online, the plots will not be interactive. Run the code locally to try it out.

:::

If [hvplot](https://github.com/holoviz/hvplot) is installed, we can visualize the fit results interactively with the {class}`qshow <erlab.accessors.general.InteractiveDatasetAccessor>` accessor.

To plot the data with the fit and fit components:

In [ ]:
result_ds.qshow(plot_components=True)

To plot each parameter as a function of the coordinate:

In [ ]:
result_ds.qshow.params()

### Parallelization

The accessors are tightly integrated with `xarray`, so passing a dask array will parallelize the fitting process. See [Parallel Computing with Dask](https://docs.xarray.dev/en/stable/user-guide/dask.html) for more information.

For non-dask objects, you can achieve [joblib](https://joblib.readthedocs.io/en/stable/)-based parallelization:

- For non-dask Datasets, basic parallelization across multiple data variables can be achieved with the `parallel` argument to {meth}`xarray.Dataset.modelfit <erlab.accessors.fit.ModelFitDatasetAccessor.__call__>`.

- For parallelizing fits on a single DataArray along a dimension with many points, the {meth}`xarray.DataArray.parallel_fit <erlab.accessors.fit.ParallelFitDataArrayAccessor>` accessor can be used. This method is similar to {meth}`xarray.DataArray.modelfit <erlab.accessors.fit.ModelFitDataArrayAccessor.__call__>`, but requires the name of the dimension to parallelize over instead of the coordinates to fit along. For example, to parallelize the fit in the previous example, you can use the following code:

    ```python

    gold_selected.parallel_fit(
        dim="alpha",
        model=FermiEdgeModel(),
        params={"temp": {"value": 100.0, "vary": False}},
        guess=True,
    )
    ```

    :::{note}
  
    - Note that the initial run will take a long time due to the overhead of creating parallel workers. Subsequent calls will run faster, since joblib's default backend will try to reuse the workers.
      
    - The accessor has some intrinsic overhead due to post-processing. If you need the best performance, handle the parallelization yourself with joblib and {meth}`lmfit.Model.fit <lmfit.model.Model.fit>`.

    :::


### Saving and loading fits

Since the resulting dataset contains no Python objects, it can be saved and loaded reliably as a netCDF file using {meth}`xarray.Dataset.to_netcdf` and {func}`xarray.open_dataset`, respectively. If you wish to obtain the {class}`lmfit.model.ModelResult` objects from the fit, you can use the `output_result` argument.

:::{warning}

    Saving full model results that includes the model functions can be difficult. Instead of saving the fit results, it is recommended to save the code that can reproduce the fit. See [the relevant lmfit documentation](https://lmfit.github.io/lmfit-py/model.html#saving-and-loading-modelresults) for more information.

:::

## Using `iminuit`

:::{note}

This part requires the optional [iminuit](https://github.com/scikit-hep/iminuit) dependency.

:::

[iminuit](https://github.com/scikit-hep/iminuit) is a powerful Python interface to the [Minuit C++ library](https://root.cern.ch/doc/master/Minuit2Page.html) developed at CERN. To learn more, see the [iminuit documentation](http://scikit-hep.org/iminuit/).

ERLabPy provides a thin wrapper around {class}`iminuit.Minuit` that allows you to use lmfit models with iminuit. The example below conducts the same fit as the previous one, but using iminuit.

In [ ]:
model = era.fit.models.MultiPeakModel(
    npeaks=2, peak_shapes=["lorentzian"], fd=False, convolve=True
)

m = era.fit.minuit.Minuit.from_lmfit(
    model,
    mdc,
    mdc.kx,
    p0_center=-0.5,
    p1_center=0.5,
    p0_width=0.03,
    p1_width=0.03,
    p0_height=1000,
    p1_height=1000,
    lin_bkg={"value": 0.0, "vary": False},
    const_bkg=0.0,
    resolution=0.03,
)

m.migrad()
m.minos()
m.hesse()

You can also use the [interactive fitting interface](https://scikit-hep.org/iminuit/notebooks/interactive.html) provided by iminuit.

:::{note}

- Requires [ipywidgets](https://github.com/jupyter-widgets/ipywidgets) to be installed.
    
- If you are viewing this documentation online, changing the sliders won’t change the plot. run the code locally to try it out.

:::

In [ ]:
m.interactive()